In [1]:
from pandas_datareader import wb
import pandas as pd
import warnings

In [2]:
warnings.simplefilter(action='ignore', category=FutureWarning)

In [36]:
wskazniki = {
    # To jest CAŁOŚĆ inwestycji w kraju (budowa fabryk, dróg, maszyn)
    'NE.GDI.FTOT.ZS': 'Total_Investment_GDP', 
    
    # To już masz (FDI), ale przypominam:
    'BX.KLT.DINV.WD.GD.ZS': 'FDI_Inflows'
}

df = wb.download(indicator=wskazniki, country='all', start=2018, end=2025)

df = df.rename(columns=wskazniki)
df = df.reset_index()
df = df.groupby('country')[list(wskazniki.values())].last()

if 'country' not in df.columns:
    df = df.reset_index()

In [37]:
df.info()
df['Domestic_Investment'] = df['Total_Investment_GDP'] - df['FDI_Inflows']
df['Foreign_Dependency_Ratio'] = df['FDI_Inflows'] / df['Total_Investment_GDP']

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 266 entries, 0 to 265
Data columns (total 3 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   country               266 non-null    object 
 1   Total_Investment_GDP  213 non-null    float64
 2   FDI_Inflows           248 non-null    float64
dtypes: float64(2), object(1)
memory usage: 6.4+ KB


In [19]:
#ekonomiczne 
wskazniki = {
    # Ogólne
    'NY.GDP.PCAP.PP.CD': 'GDP_PerCapita_PPP',
    'NY.GDP.MKTP.KD.ZG': 'GDP_Growth',
    'FP.CPI.TOTL.ZG': 'Inflation',
    
    # Struktura gospodarki
    'NV.AGR.TOTL.ZS': 'Agriculture_Share_GDP',   # Dużo = Biedny
    'NV.IND.TOTL.ZS': 'Industry_Share_GDP',      # Przemysł
    'NV.SRV.TOTL.ZS': 'Services_Share_GDP',      # Usługi (Dużo = Bogaty)
    
    # Rynek pracy i Zależności
    'SL.UEM.TOTL.ZS': 'Unemployment_Rate',       # Bezrobocie
    'BX.TRF.PWKR.DT.GD.ZS': 'Remittances_GDP',   # Przelewy od emigrantów (Zależność od zagranicy)
    
    # Zaawansowanie
    'TX.VAL.TECH.MF.ZS': 'High_Tech_Exports',    # Innowacyjność
    'BX.KLT.DINV.WD.GD.ZS': 'FDI_Inflows'        # Inwestycje zagraniczne
}    

In [3]:
#education
wskazniki = {
    # --- TWOJE OBECNE (Baza) ---
    'SE.TER.ENRR': 'Tertiary_Enrollment',        # Partycypacja w edukacji wyższej (Kluczowe dla krajów bogatych)
    'IT.NET.USER.ZS': 'Internet_Usage',          # Dostęp do wiedzy/cyfryzacja (Nowoczesna edukacja)
    'IP.PAT.NRES': 'Patents_NonResidents',       # Czy zagranica chce tu patentować? 
    'IP.JRN.ARTC.SC': 'Scientific_Articles',     # Ilość produkowanych artykułów

    # --- WYNIKI (Outcome - co jest Twoim 'y'?) ---
    'SE.PRM.CMPT.ZS': 'Primary_Completion_Rate', # Jaki % dzieci kończy szkołę podstawową (Kluczowe dla krajów rozwijających się)

    # --- NAKŁADY (Input - ile inwestują?) ---
    'SE.XPD.TOTL.GD.ZS': 'Gov_Expenditure_Educ_GDP', # Wydatki rządowe na edukację jako % PKB

    # --- JAKOŚĆ I WARUNKI (Process) ---
    'SE.SEC.ENRR': 'Secondary_Enrollment',       # Partycypacja w szkołach średnjih (Pomost między podstawówką a studiami)
    'SE.COM.DURS': 'Compulsory_Years',           # Ile lat nauki jest prawnie obowiązkowe?
}

In [17]:
#healthcare
wskazniki = {
    # --- WYNIKI (To co już masz - Outcomes) ---
    'SP.DYN.LE00.IN': 'Life_Expectancy',           # Oczekiwana długość życia (Twój główny cel - y)
    'SP.DYN.IMRT.IN': 'Infant_Mortality',          # Śmiertelność niemowląt (na 1000 urodzeń)
    'SP.DYN.TFRT.IN': 'Fertility_Rate',            # Dzietność
    'SH.DYN.MORT': 'Mortality_rate_under_5',       # Śmiertelność dzieci do lat 5

    # --- ZASOBY I FINANSE (Ile kraj inwestuje?) ---
    'SP.POP.65UP.TO.ZS': 'Population_Ages_65_plus',# Odsetek seniorów (Wysoki w Developed, niski w Struggling)
    'SH.XPD.CHEX.GD.ZS': 'Health_Expenditure_GDP', # Wydatki na zdrowie jako % PKB (Kluczowe!)
    'SH.XPD.OOPC.CH.ZS': 'Out_of_Pocket_Exp',      # Ile ludzie płacą sami z kieszeni (% wydatków)
    
    # --- RYZYKO I CHOROBY (Obciążenia systemu) ---
    'SP.POP.DPND': 'Age_Dependency_Ratio'          # Współczynnik obciążenia demograficznego (starzy/młodzi vs pracujący)
}

In [23]:
#social
wskazniki = {
    # --- TWOJE OBECNE (Baza: Miasto vs Wieś) ---
    'SP.URB.TOTL.IN.ZS': 'Urbanization_Rate',         # Wskaźnik urbanizacji (% ludzi w miastach) - Świetny kandydat na cel (y)!
    'SP.RUR.TOTL.ZG': 'Rural_Population_Growth',      # Wzrost populacji wiejskiej (często ujemny w krajach rozwiniętych)
    'SL.AGR.EMPL.ZS': 'Employment_In_Agriculture',    # % ludzi pracujących w rolnictwie (Lepsze społecznie niż PKB z rolnictwa)

    # --- DEMOGRAFIA (Struktura wieku) ---
    'SP.POP.65UP.TO.ZS': 'Population_Ages_65_and_above', # Starzenie się społeczeństwa (Wysokie w UE/Japonii)
    'SP.POP.0014.TO.ZS': 'Population_Ages_0_14',         # Odsetek dzieci (Wysoki w krajach rozwijających się)
    'SP.POP.DPND': 'Age_Dependency_Ratio',               # Obciążenie demograficzne (ile osób niepracujących na pracującego)

    # --- RÓWNOŚĆ I PŁEĆ (Inclusion) ---
    'SG.GEN.PARL.ZS': 'Women_In_Parliament',          # Kobiety w parlamencie (%) - Miara równouprawnienia
    'SL.TLF.CACT.FE.ZS': 'Female_Labor_Participation', # Aktywność zawodowa kobiet

    # --- JAKOŚĆ ŻYCIA I BEZPIECZEŃSTWO ---
    'EG.ELC.ACCS.ZS': 'Access_to_Electricity',        # Dostęp do prądu (% populacji)
    'IT.CEL.SETS.P2': 'Mobile_Subscriptions',         # Liczba telefonów na 100 osób (często >100)
    'VC.IHR.PSRC.P5': 'Intentional_Homicides',        # Liczba zabójstw na 100k (Wskaźnik bezpieczeństwa/przestępczości)
}

In [9]:
#infrastructure
wskazniki = {
    # --- FUNDAMENTY (To oddzieli Struggling od reszty) ---
    'EG.ELC.ACCS.ZS': 'Access_to_Electricity',       # Dostęp do prądu (% populacji)
    'SH.H2O.SMDW.ZS': 'Access_to_Safe_Water',        # Dostęp do bezpiecznej wody pitnej (Kluczowa infrastruktura sanitarna)

    # --- ŁĄCZNOŚĆ CYFROWA (To oddzieli Developing od Developed) ---
    'IT.NET.USER.ZS': 'Internet_Usage',              # Procent użytkowników Internetu
    'IT.NET.BBND.P2': 'Fixed_Broadband_Subs',        # Stałe łącza szerokopasmowe na 100 osób (To jest "autostrada XXI wieku")
    
    # --- TRANSPORT I ENERGIA (To zidentyfikuje HUBY) ---
    'IS.AIR.PSGR': 'Air_Transport_Passengers',       # Liczba pasażerów lotniczych (Globalne Huby mają tu miliony)
    'EG.USE.ELEC.KH.PC': 'Electric_Power_Consumption', # Zużycie prądu na osobę (Miernik uprzemysłowienia)
    
    # --- NOWOCZESNOŚĆ ---
    'EG.FEC.RNEW.ZS': 'Renewable_Energy_Share'       # Udział energii odnawialnej (Często wysoki w bogatych Hubach LUB biednych krajach z hydroenergią)
}

In [5]:
df = wb.download(indicator=wskazniki, country='all', start=2005, end=2025)

ReadTimeout: HTTPSConnectionPool(host='api.worldbank.org', port=443): Read timed out. (read timeout=30)

In [25]:
df = df.rename(columns=wskazniki)
df = df.reset_index()
df = df.groupby('country')[list(wskazniki.values())].last()

In [26]:
if 'country' not in df.columns:
    df = df.reset_index()

In [27]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 266 entries, 0 to 265
Data columns (total 12 columns):
 #   Column                        Non-Null Count  Dtype  
---  ------                        --------------  -----  
 0   country                       266 non-null    object 
 1   Urbanization_Rate             265 non-null    float64
 2   Rural_Population_Growth       254 non-null    float64
 3   Employment_In_Agriculture     235 non-null    float64
 4   Population_Ages_65_and_above  265 non-null    float64
 5   Population_Ages_0_14          265 non-null    float64
 6   Age_Dependency_Ratio          265 non-null    float64
 7   Women_In_Parliament           241 non-null    float64
 8   Female_Labor_Participation    235 non-null    float64
 9   Access_to_Electricity         263 non-null    float64
 10  Mobile_Subscriptions          258 non-null    float64
 11  Intentional_Homicides         241 non-null    float64
dtypes: float64(11), object(1)
memory usage: 25.1+ KB


In [28]:
df = df.dropna()
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 207 entries, 0 to 265
Data columns (total 12 columns):
 #   Column                        Non-Null Count  Dtype  
---  ------                        --------------  -----  
 0   country                       207 non-null    object 
 1   Urbanization_Rate             207 non-null    float64
 2   Rural_Population_Growth       207 non-null    float64
 3   Employment_In_Agriculture     207 non-null    float64
 4   Population_Ages_65_and_above  207 non-null    float64
 5   Population_Ages_0_14          207 non-null    float64
 6   Age_Dependency_Ratio          207 non-null    float64
 7   Women_In_Parliament           207 non-null    float64
 8   Female_Labor_Participation    207 non-null    float64
 9   Access_to_Electricity         207 non-null    float64
 10  Mobile_Subscriptions          207 non-null    float64
 11  Intentional_Homicides         207 non-null    float64
dtypes: float64(11), object(1)
memory usage: 21.0+ KB


In [ ]:
# Lista nazw, które oznaczamy jako "Region"
lista_regionow = [
    'Africa Eastern and Southern', 'Africa Western and Central', 'Arab World',
    'Caribbean small states', 'Central Europe and the Baltics', 'Early-demographic dividend',
    'East Asia & Pacific', 'East Asia & Pacific (IDA & IBRD countries)',
    'East Asia & Pacific (excluding high income)', 'Euro area', 'Europe & Central Asia',
    'Europe & Central Asia (IDA & IBRD countries)', 'Europe & Central Asia (excluding high income)',
    'European Union', 'Fragile and conflict affected situations',
    'Heavily indebted poor countries (HIPC)', 'High income', 'IBRD only',
    'IDA & IBRD total', 'IDA blend', 'IDA only', 'IDA total', 'Late-demographic dividend',
    'Latin America & Caribbean', 'Latin America & Caribbean (excluding high income)',
    'Latin America & the Caribbean (IDA & IBRD countries)',
    'Least developed countries: UN classification', 'Low & middle income', 'Low income',
    'Lower middle income', 'Middle East, North Africa, Afghanistan & Pakistan',
    'Middle East, North Africa, Afghanistan & Pakistan (IDA & IBRD)',
    'Middle East, North Africa, Afghanistan & Pakistan (excluding high income)',
    'Middle income', 'North America', 'OECD members', 'Other small states',
    'Pacific island small states', 'Post-demographic dividend', 'Pre-demographic dividend',
    'Small states', 'South Asia', 'South Asia (IDA & IBRD)', 'Sub-Saharan Africa',
    'Sub-Saharan Africa (IDA & IBRD countries)', 'Sub-Saharan Africa (excluding high income)',
    'Upper middle income', 'World', 'Not classified'
]

df['Category'] = df['country'].apply(lambda x: 'Region' if x in lista_regionow else 'Country')

print(df['Category'].value_counts())

df.to_csv('data/core-peryferia/peryferia.csv', index=False, sep=',', decimal='.')

Category
Country    217
Region      49
Name: count, dtype: int64


In [15]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 266 entries, 0 to 265
Data columns (total 9 columns):
 #   Column                      Non-Null Count  Dtype  
---  ------                      --------------  -----  
 0   country                     266 non-null    object 
 1   Access_to_Electricity       263 non-null    float64
 2   Access_to_Safe_Water        179 non-null    float64
 3   Internet_Usage              242 non-null    float64
 4   Fixed_Broadband_Subs        256 non-null    float64
 5   Air_Transport_Passengers    220 non-null    float64
 6   Electric_Power_Consumption  194 non-null    float64
 7   Renewable_Energy_Share      260 non-null    float64
 8   Category                    266 non-null    object 
dtypes: float64(7), object(2)
memory usage: 18.8+ KB


In [16]:
kolumny = df.columns.tolist()

kolumny.remove('Category')
kolumny.insert(1, 'Category')

df = df[kolumny]
df['Category'] = df['Category'].astype('category')

df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 240 entries, 0 to 265
Data columns (total 10 columns):
 #   Column                   Non-Null Count  Dtype   
---  ------                   --------------  -----   
 0   country                  240 non-null    object  
 1   Category                 240 non-null    category
 2   Life_Expectancy          240 non-null    float64 
 3   Infant_Mortality         240 non-null    float64 
 4   Fertility_Rate           240 non-null    float64 
 5   Mortality_rate_under_5   240 non-null    float64 
 6   Population_Ages_65_plus  240 non-null    float64 
 7   Health_Expenditure_GDP   240 non-null    float64 
 8   Out_of_Pocket_Exp        240 non-null    float64 
 9   Age_Dependency_Ratio     240 non-null    float64 
dtypes: category(1), float64(8), object(1)
memory usage: 19.1+ KB


In [ ]:
wskazniki_full = {
    # 1. Gospodarka
    'NY.GDP.PCAP.PP.CD': 'GDP_Per_Capita_PPP',
    'NY.GDP.MKTP.KD.ZG': 'GDP_Growth',
    'NE.TRD.GNFS.ZS': 'Trade_percent_GDP',
    'DT.DOD.DECT.GN.ZS': 'External_Debt_GNI',
    'FP.CPI.TOTL.ZG': 'Inflation',

    # 2. Biznes i Finanse
    'FS.AST.PRVT.GD.ZS': 'Domestic_Credit_Private',
    'BX.KLT.DINV.WD.GD.ZS': 'FDI_Inflows',
    'IC.REG.DURS': 'Time_Start_Business',
    'IC.TAX.TOTL.CP.ZS': 'Tax_Rate_Commercial',
    'DT.ODA.ODAT.PC.ZS': 'Aid_Received_Per_Capita',

    # 3. Edukacja i Nauka
    'GB.XPD.RSDV.GD.ZS': 'RD_Expenditure',
    'SP.POP.SCIE.RD.P6': 'Researchers_per_Million',
    'SE.TER.ENRR': 'Tertiary_Enrollment',
    'SE.ADT.LITR.ZS': 'Literacy_Rate',
    'TX.VAL.TECH.MF.ZS': 'High_Tech_Exports',

    # 4. Zdrowie
    'SP.DYN.LE00.IN': 'Life_Expectancy',
    'SP.DYN.IMRT.IN': 'Infant_Mortality',
    'SH.XPD.CHEX.GD.ZS': 'Health_Expenditure',
    'SP.DYN.TFRT.IN': 'Fertility_Rate',
    'SG.GEN.PARL.ZS': 'Women_In_Parliament',

    # 5. Spoleczenstwo
    'SI.POV.GINI': 'Gini_Index',
    'SI.POV.DDAY': 'Poverty_Headcount_2_15',
    'SI.DST.05TH.20': 'Income_Share_Lowest_20',
    'SL.UEM.1524.ZS': 'Youth_Unemployment',
    'SL.TLF.0714.ZS': 'Child_Labor',

    # 6. Infrastruktura
    'EG.ELC.ACCS.ZS': 'Electricity_Access',
    'IT.NET.USER.ZS': 'Internet_Usage',
    'SP.URB.TOTL.IN.ZS': 'Urbanization_Rate',
    'SH.H2O.SMDW.ZS': 'Safe_Water_Access',
    'NV.AGR.TOTL.ZS': 'Agriculture_Share_GDP',

    # 7. Srodowisko
    'EN.ATM.CO2E.PC': 'CO2_Emissions_Per_Capita',
    'EG.USE.PCAP.KG.OE': 'Energy_Use_Per_Capita',
    'EG.ELC.RNEW.ZS': 'Renewable_Energy_Output',
    'AG.LND.FRST.ZS': 'Forest_Area',
    'EN.ATM.PM25.MC.M3': 'PM2_5_Pollution'
}